In [9]:
# Needed if running on Colab
!pip3 install open-spiel
!pip3 install torch


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [50]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import random
from go_search_problem import GoProblem, GoState
from heuristic_go_problems import GoProblemLearnedHeuristic, GoProblemSimpleHeuristic
from agents import GreedyAgent, RandomAgent, MCTSAgent, GameAgent, MinimaxAgent, AlphaBetaAgent, IterativeDeepeningAgent
import matplotlib.pyplot as plt
from tqdm import tqdm
from game_runner import run_many
import pickle

torch.set_default_tensor_type(torch.FloatTensor)

In [11]:
def load_dataset(path: str):
    with open(path, 'rb') as f:
        dataset = pickle.load(f)
    return dataset

dataset_5x5 = load_dataset('dataset_5x5.pkl')
# dataset_9x9 = load_dataset('9x9_dataset.pkl')

In [12]:
def save_model(path: str, model):
    """
    Save model to a file
    Input:
        path: path to save model to
        model: Pytorch model to save
    """
    torch.save({
        'model_state_dict': model.state_dict(),
    }, path)

def load_model(path: str, model):
    """
    Load model from file

    Note: you still need to provide a model (with the same architecture as the saved model))

    Input:
        path: path to load model from
        model: Pytorch model to load
    Output:
        model: Pytorch model loaded from file
    """
    checkpoint = torch.load(path)
    model.load_state_dict(checkpoint['model_state_dict'])
    return model

# Task 1: Convert GameState to Features

In [13]:
def get_features(game_state: GoState):
    """
    Map a game state to a list of features.

    Some useful functions from game_state include:
        game_state.size: size of the board
        get_pieces_coordinates(player_index): get coordinates of all pieces of a player (0 or 1)
        get_pieces_array(player_index): get a 2D array of pieces of a player (0 or 1)
        
        get_board(): get a 2D array of the board with 4 channels (player 0, player 1, empty, and player to move). 4 channels means the array will be of size 4 x n x n
    
        Descriptions of these methods can be found in the GoState

    Input:
        game_state: GoState to encode into a fixed size list of features
    Output:
        features: list of features
    """
    board_size = game_state.size
    # black_pieces = game_state.get_pieces_array(0).flatten()
    # white_pieces = game_state.get_pieces_array(1).flatten()

    # One-hot encoding of stone locations (n * n * 2 features)
    # stone_locations = np.concatenate((black_pieces, white_pieces)).tolist()

    # Player to move ( 0 = black, 1 = white)
    # player_to_move = game_state.player_to_move()


    # TODO: Encode game_state into a list of features
    features = [] 
    # features.extend(board)
    # features.append(player_to_move)
    features = np.array(game_state.get_board()).flatten()
    return features

In [14]:
# Print information about first data point
data_point = dataset_5x5[0]
features = get_features(data_point[0])
action = data_point[1]
result = data_point[2]
print(data_point[0])
print("Number of features: ", len(features))
print("features", features)
print("Action #", action)
print("Game Result", result)

GoState(komi=0.5, to_play=B, history.size()=0)

 5 +++++
 4 +++++
 3 +++++
 2 +++++
 1 +++++
   ABCDE

Number of features:  100
features [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0.]
Action # 11
Game Result -1.0


# Task 2: Supervised Learning of a Value Network

In [15]:
class ValueNetwork(nn.Module):
    def __init__(self, input_size):
      super(ValueNetwork, self).__init__()

      # TODO: What should the output size of a Value function be?
      output_size = 1

      # TODO: Add more layers, non-linear functions, etc.=
      self.linear = nn.Linear(input_size, output_size)
      
      # self.fc1 = nn.Linear(input_size, 128)
      # self.fc2 = nn.Linear(128, 64)
      # self.fc3 = nn.Linear(64, 32)
      # self.fc4 = nn.Linear(32, 16)
      # self.fc5 = nn.Linear(16, output_size)
      
      self.fc1 = nn.Linear(input_size, 32)
      self.fc2 = nn.Linear(32, 16)
      self.fc3 = nn.Linear(16, 8)
      self.fc4 = nn.Linear(8, output_size)

      self.sigmoid = nn.Sigmoid()
      self.tanh = nn.Tanh()
      self.relu = nn.ReLU() 
      


    def forward(self, x):
      """
      Run forward pass of network

      Input:
        x: input to network
      Output:
        output of network
      """
      # TODO: Update as more layers are added

      z1 = self.fc1(x)
      a1 = self.relu(z1)
      
      z2 = self.fc2(a1)
      a2 = self.tanh(z2)
      
      z3 = self.fc3(a2)
      a3 = self.relu(z3)

      z4 = self.fc4(a3)
      # a4 = self.tanh(z4)
      output = self.sigmoid(z4)

      # z5 = self.fc5(a4)
      # output = self.sigmoid(z5)

      return output

In [16]:
# This will not produce meaningful outputs until trained, but you can test for syntax errors
features_tensor = torch.Tensor(features)
value_net = ValueNetwork(len(features))
print("predicted Value", value_net(features_tensor))

predicted Value tensor([0.4719], grad_fn=<SigmoidBackward0>)


In [444]:
def train_value_network(dataset, num_epochs, learning_rate):
    """
    Train a value network on the provided dataset.

    Input:
        dataset: list of (state, action, result) tuples
        num_epochs: number of epochs to train for
        learning_rate: learning rate for gradient descent
    Output:
        model: trained model
    """
    # Make sure dataset is shuffled for better performance
    random.shuffle(dataset)
    
    # You may find it useful to create train/test sets to better track performance/overfit/underfit
    # train_size = int(0.8 * len(dataset))
    # train_data = dataset[:train_size]
    # test_data = dataset[train_size:]

    # TODO: Create model
    input_size = len(get_features(dataset[0][0]))
    model = ValueNetwork(input_size)

    # TODO: Specify Loss Function - use MSE bc continuous outcome value
    loss_function = nn.MSELoss()

    # You can use Adam, which is stochastic gradient descent with ADAptive Momentum
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    batch_size = 32

    for epoch in range(num_epochs):

        batch_loss = 0.0
        batch_counter = 0

        # state = (state, action, outcome)
        for data_point in dataset:
            state = data_point[0]
            features = get_features(state)
            features_tensor = torch.tensor(features, dtype=torch.float32)

            # TODO: What should the desired output of the value network be?
            # Note: You will have to convert the label to a torch tensor to use with torch's loss functions
            outcome = data_point[2]
            label = torch.tensor(outcome)

            # TODO: Get model prediction of value
            prediction = model(features_tensor)

            # TODO: Compute Loss for data point
            loss = loss_function(prediction, label)
            batch_loss += loss
            batch_counter += 1

            if batch_counter % batch_size == 0:
                # Call backward to run backward pass and compute gradients
                batch_loss.backward()

                # Run gradient descent step with optimizer
                optimizer.step()

                # Reset gradient for next batch
                optimizer.zero_grad()
                batch_loss = 0.0
        print("Epoch: ", epoch)
        print("Loss: ", batch_loss)
    return model

value_model = train_value_network(dataset_5x5, 10, 1e-4)
save_model("value_model_2.pt", value_model)

Epoch:  0
Loss:  tensor(4.2536, grad_fn=<AddBackward0>)
Epoch:  1
Loss:  tensor(4.0805, grad_fn=<AddBackward0>)
Epoch:  2
Loss:  tensor(4.0346, grad_fn=<AddBackward0>)
Epoch:  3
Loss:  tensor(4.0134, grad_fn=<AddBackward0>)
Epoch:  4
Loss:  tensor(4.0152, grad_fn=<AddBackward0>)
Epoch:  5
Loss:  tensor(4.0229, grad_fn=<AddBackward0>)
Epoch:  6
Loss:  tensor(4.0310, grad_fn=<AddBackward0>)
Epoch:  7
Loss:  tensor(4.0374, grad_fn=<AddBackward0>)
Epoch:  8
Loss:  tensor(4.0384, grad_fn=<AddBackward0>)
Epoch:  9
Loss:  tensor(4.0453, grad_fn=<AddBackward0>)


## Comparing Learned Value function against other Agents

In [56]:
class GoProblemLearnedHeuristic(GoProblem):
    def __init__(self, model=None, state=None):
        super().__init__(state=state)
        self.model = model
        
    def __call__(self, model=None):
        """
        Use the model to compute a heuristic value for a given state.
        """
        return self

    def encoding(self, state):
        """
        Get encoding of state (convert state to features)
        Note, this may call get_features() from Task 1. 

        Input:
            state: GoState to encode into a fixed size list of features
        Output:
            features: list of features
        """
        # TODO: get encoding of state (convert state to features)
        features = get_features(state)

        return features

    def heuristic(self, state, player_index):
        """
        Return heuristic (value) of current state

        Input:
            state: GoState to encode into a fixed size list of features
            player_index: index of player to evaluate heuristic for
        Output:
            value: heuristic (value) of current state
        """
        # TODO: Compute heuristic (value) of current state
        features = self.encoding(state)
        features_tensor = torch.tensor(features, dtype=torch.float32)
        value = self.model(features_tensor)
        
        # if player_index == 1:
        #     value = -value

        # Note, your agent may perform better if you force it not to pass
        # (i.e., don't select action #25 on a 5x5 board unless necessary)
        return value

    def __str__(self) -> str:
        return "Learned Heuristic"


def create_value_agent_from_model():
    """
    Create agent object from saved model. This (or other methods like this) will be how your agents will be created in gradescope and in the final tournament.
    """

    model_path = "value_model.pt"
    # TODO: Update number of features for your own encoding size
    feature_size =  len(features)
    model = load_model(model_path, ValueNetwork(feature_size))
    heuristic_search_problem = GoProblemLearnedHeuristic(model)

    # TODO: Try with other heuristic agents (IDS/AB/Minimax)
    learned_agent = GreedyAgent(heuristic_search_problem)

    return learned_agent

learned_agent = create_value_agent_from_model()
agent2 = GreedyAgent()
print("Minimax Agent", agent2)
print("Learned Minimax Agent", learned_agent)

run_many(learned_agent, agent2, 100)

Minimax Agent GreedyAgent + Simple Heuristic
Learned Minimax Agent GreedyAgent + Learned Heuristic


100%|██████████| 50/50 [00:03<00:00, 14.90it/s]

Agent 1: GreedyAgent + Learned Heuristic Score: 100.0
Agent 2: GreedyAgent + Simple Heuristic Score: -100.0
Agent 1: GreedyAgent + Learned Heuristic Score with Black (first move): 50.0
Agent 2: GreedyAgent + Simple Heuristic Score with Black (first move): -50.0
Agent 1: GreedyAgent + Learned Heuristic Average Duration: 0.001162473041132876
Agent 2: GreedyAgent + Simple Heuristic Average Duration: 0.00035214046679045045
Agent 1: GreedyAgent + Learned Heuristic Longest Duration: 0.003551006317138672
Agent 2: GreedyAgent + Simple Heuristic Longest Duration: 0.0010340213775634766
Agent 1: GreedyAgent + Learned Heuristic Average Time Remaining: 36.97482639074325
Agent 2: GreedyAgent + Simple Heuristic Average Time Remaining: 36.99227107048036
Agent 1: GreedyAgent + Learned Heuristic Min Time Remaining: 33.97310280799866
Agent 2: GreedyAgent + Simple Heuristic Min Time Remaining: 33.99274182319641


(100.0, -100.0)

# Task 3: Supervised Learning of a Policy Network

In [582]:
class PolicyNetwork(nn.Module):
    def __init__(self, input_size, board_size=5):
      super(PolicyNetwork, self).__init__()

      # TODO: What should the output size of the Policy be?
      output_size = (board_size * board_size) + 1

      # TODO: Add more layers, non-linear functions, etc.
      self.linear = nn.Linear(input_size, output_size)
      
      self.fc1 = nn.Linear(input_size, 32)
      self.fc2 = nn.Linear(32, 16)
      self.fc3 = nn.Linear(16, 8)
      self.fc4 = nn.Linear(8, output_size)

      self.sigmoid = nn.Sigmoid()
      self.tanh = nn.Tanh()
      self.relu = nn.ReLU() 
      self.softmax = nn.Softmax(dim=-1) 
      # needed for class probabilities; output probabilities sum to 1

    def forward(self, x):
      # TODO: Update as more layers are added

      z1 = self.fc1(x)
      a1 = self.relu(z1)
      
      z2 = self.fc2(a1)
      a2 = self.relu(z2)
      
      z3 = self.fc3(a2)
      a3 = self.relu(z3)

      z4 = self.fc4(a3)
      output = self.sigmoid(z4)

      return output

In [583]:
# This will not produce meaningful outputs until trained, but you can test for syntax errors
features_tensor = torch.Tensor(features)
policy_net = PolicyNetwork(len(features))
print("Predicted Action Probabilities", policy_net(features_tensor))

Predicted Action Probabilities tensor([0.5614, 0.5655, 0.4702, 0.4453, 0.4937, 0.5058, 0.4893, 0.5743, 0.5265,
        0.5309, 0.5540, 0.4538, 0.4300, 0.4554, 0.5445, 0.5680, 0.4971, 0.4439,
        0.5606, 0.5611, 0.5164, 0.4933, 0.5708, 0.5643, 0.5555, 0.4258],
       grad_fn=<SigmoidBackward0>)


In [584]:
def train_policy_network(dataset, num_epochs, learning_rate):
    """
    Train a policy network on the provided dataset.

    Input:
        dataset: list of (state, action, result) tuples
        num_epochs: number of epochs to train for
        learning_rate: learning rate for gradient descent
    Output:
        model: trained model
    """
    random.shuffle(dataset)

    # TODO: Create model
    feature_size =  len(get_features(dataset[0][0]))
    model = PolicyNetwork(feature_size)

    # TODO: Specify Loss Function
    loss_function = nn.CrossEntropyLoss()

    # You can use Adam, which is stochastic gradient descent with ADAptive Momentum
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    batch_size = 32

    for epoch in range(num_epochs):

        # batch_loss = 0.0
        # batch_counter = 0

        # data = (state, action, outcome)
        for data_point in dataset:
            optimizer.zero_grad()

            # TODO: Get features from state and convert features to torch tensor
            state = data_point[0]
            action = data_point[1]

            features = get_features(state)
            features_tensor = torch.tensor(features, dtype=torch.float32)

            # TODO: What should the desired output of the value network be?
            # Note: You will have to convert the label to a torch tensor to use with torch's loss functions
            label = torch.tensor(action)

            # TODO: Get model estimate of value
            prediction = model(features_tensor)

            # TODO: Compute Loss for data point
            loss = loss_function(prediction, label)
            # batch_loss += loss
            # batch_counter += 1

            # if batch_counter % batch_size == 0:
            #     # Call backward to run backward pass and compute gradients
            #     batch_loss.backward()

            #     # Run gradient descent step with optimizer
            #     optimizer.step()

            #     # print("Batch_Loss: ", batch_loss)
            #     optimizer.zero_grad()
            #     batch_loss = 0.0

            loss.backward()

            optimizer.step()

        print("Epoch: ", epoch)
        print("Loss: ", loss)
        
    return model

policy_net = train_policy_network(dataset_5x5, 10, 1e-4)
save_model("policy_model_3.pt", policy_net)

Epoch:  0
Loss:  tensor(2.3527, grad_fn=<NllLossBackward0>)
Epoch:  1
Loss:  tensor(2.3630, grad_fn=<NllLossBackward0>)
Epoch:  2
Loss:  tensor(2.4049, grad_fn=<NllLossBackward0>)
Epoch:  3
Loss:  tensor(2.4167, grad_fn=<NllLossBackward0>)
Epoch:  4
Loss:  tensor(2.4200, grad_fn=<NllLossBackward0>)
Epoch:  5
Loss:  tensor(2.4192, grad_fn=<NllLossBackward0>)
Epoch:  6
Loss:  tensor(2.4185, grad_fn=<NllLossBackward0>)
Epoch:  7
Loss:  tensor(2.4185, grad_fn=<NllLossBackward0>)
Epoch:  8
Loss:  tensor(2.4170, grad_fn=<NllLossBackward0>)
Epoch:  9
Loss:  tensor(2.4027, grad_fn=<NllLossBackward0>)


## Comparing Learned Policy against other Agents

In [602]:
class PolicyAgent(GameAgent):
    def __init__(self, search_problem, model_path, board_size=5):
        super().__init__()
        self.search_problem = search_problem
        # self.model = load_model(model_path, PolicyNetwork)
        self.board_size = board_size

        feature_size = board_size ** 2 * 4
        self.model = load_model(model_path, PolicyNetwork(feature_size, board_size))
        self.model.eval()

    def encoding(self, state):
        # TODO: get encoding of state (convert state to features)
        features = get_features(state)

        return features

    def get_move(self, game_state, time_limit=1):
      """
      Get best action for current state using self.model

      Input:
        game_state: current state of the game
        time_limit: time limit for search (This won't be used in this agent)
      Output:
        action: best action to take
      """

      # TODO: Select LEGAL Best Action predicted by model

      features = self.encoding(game_state)
      features_tensor = torch.tensor(features, dtype=torch.float32)
      
      probabilities = self.model(features_tensor)
      legal_actions = self.search_problem.get_available_actions(game_state)

      # The top prediction of your model may not be a legal move!
      best_action = random.choice(legal_actions)
      action_probabilities = {}

      for action in legal_actions:
          action_probabilities[action] = probabilities[action]

      best_prob = float('-inf') 

      for action, prob in action_probabilities.items(): 
          if prob > best_prob: 
              best_prob = prob
              best_action = action

      if best_action == 25: 
          action_probabilities[best_action] -= .2

      for action, prob in action_probabilities.items(): 
          if prob > best_prob: 
              best_prob = prob
              best_action = action

      # Note, you may want to force your policy not to pass their turn unless necessary
      assert best_action in self.search_problem.get_available_actions(game_state)
      
      return best_action

    def __str__(self) -> str:
        return "Policy Agent"
    
def create_policy_agent_from_model():
    """
    Create agent object from saved model. This (or other methods like this) will be how your agents will be created in gradescope and in the final tournament.    
    """

    model_path = "policy_model.pt"
    agent = PolicyAgent(GoProblem(size=5), model_path)
    return agent

In [611]:
# policy_agent = PolicyAgent(GoProblem(size=5), 'policy_model.pt')
policy_agent = create_policy_agent_from_model()
print("Policy Agent", policy_agent)
run_many(policy_agent, GreedyAgent(), 40)

Policy Agent Policy Agent


100%|██████████| 20/20 [00:00<00:00, 74.54it/s]

Agent 1: Policy Agent Score: 40.0
Agent 2: GreedyAgent + Simple Heuristic Score: -40.0
Agent 1: Policy Agent Score with Black (first move): 20.0
Agent 2: GreedyAgent + Simple Heuristic Score with Black (first move): -20.0
Agent 1: Policy Agent Average Duration: 0.00011111008269446236
Agent 2: GreedyAgent + Simple Heuristic Average Duration: 0.0002316193511852852
Agent 1: Policy Agent Longest Duration: 0.0013790130615234375
Agent 2: GreedyAgent + Simple Heuristic Longest Duration: 0.0014269351959228516
Agent 1: Policy Agent Average Time Remaining: 33.997926092147814
Agent 2: GreedyAgent + Simple Heuristic Average Time Remaining: 33.49577408432961
Agent 1: Policy Agent Min Time Remaining: 28.996105194091797
Agent 2: GreedyAgent + Simple Heuristic Min Time Remaining: 27.995012283325195


(40.0, -40.0)

# Submitting

After you've completed all the tasks in this notebook, you'll want to add your agents to your agents.py file. You'll want to copy the necessary function and class definitions for PolicyAgent, GoProblemLearnedHeuristic, PolicyNetwork, ValueNetwork, and any other methods you referenced. Your agents will ultimately be tested on gradescope by calling create_value_agent_from_model or by create_policy_agent_from_model.